In [100]:
from pyproj import Transformer
import requests
from osmnx.utils_geo import bbox_from_point
import time
import numpy as np
import pandas as pd

data_availability_path = "C:/Users/ckaz3/Desktop/intersection-safety-infra-detection/data/interim/data_availability.csv"
intersections = pd.read_csv(data_availability_path)

id = 11393
intersections[intersections["Intersection_ID"] == id]


,Station_ID,Agency,Line(s),Station Name,Year Opened,Station_Latitude,Station_Longitude,Inter_Latitude,Inter_Longitude,Intersection_ID,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
13474,14138,Maryland Transit Administration,Light Rail([L]),HAMBURG ST,Pre-2000,39.277641,-76.62081,39.278202,-76.619011,11393,...,1,2,2,0,2,2,0,2,0,0


In [749]:
id = 9134

intersections[intersections["Intersection_ID"] == id][["Inter_Latitude", "Inter_Longitude", "Intersection_ID"]]

,Inter_Latitude,Inter_Longitude,Intersection_ID
11145,40.752199,-73.993458,9134
31535,40.752199,-73.993458,9134


In [218]:
test = "C:/Users/ckaz3/Downloads/project-3-at-2025-07-24-14-26-5b2e8097.csv"
test = pd.read_csv(test)
print(sum(test["label"][~test["label"].isna()].str.count("daylighting")))
print(sum(test["label"][~test["label"].isna()].str.count("curb_extension")))
print(sum(test["label"][~test["label"].isna()].str.count("safety_islands")))

242
118
570


In [53]:
data_availability_path = "C:/Users/ckaz3/Desktop/intersection-safety-infra-detection/data/interim/data_availability.csv"
intersections = pd.read_csv(data_availability_path)
len(intersections)

17037

In [34]:
from io import BytesIO
import requests 
from PIL import Image

source_url = "https://cteco.uconn.edu/ctraster/rest/services/images/Ortho_2019_tiled/ImageServer/exportImage"
params = {'bbox': '-72.53808244256578,40.99221120389935,-72.53736755743421,40.992750796100644',                              
          'size': '416,416',
          'f': 'image',         
          'bboxSR': '4326',} 
source_extent = requests.get(source_url, params)
image_file_extrema = Image.open(BytesIO(source_extent.content)).getextrema()
image_file_extrema == ((0, 0), (0, 0), (0, 0), (0, 0))

True

In [ ]:
import pandas as pd

data_path = "C:/Users/ckaz3/Desktop/intersection-safety-infra-detection/data/interim/data_availability.csv"
data = pd.read_csv(data_path)
for row in data.itertuples():
    print(row.Agency)
    # print(data[data["Intersection_ID"] == row["Intersection_ID"]])
    break

ACE 


True

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

data_path = "C:/Users/ckaz3/Desktop/intersection-safety-infra-detection/data/interim/data_availability.csv"
data = pd.read_csv(data_path)


100%|██████████| 55718/55718 [00:24<00:00, 2253.91it/s]


In [164]:
data_path = "C:/Users/ckaz3/Desktop/intersection-safety-infra-detection/data/interim/intersections.csv"
data = pd.read_csv(data_path)
data["Agency"].unique()

array(['ACE ', 'ACE , Capitol Corridor Joint Powers Authority',
       'Alameda/Oakland Ferry', 'Amtrak ', 'BART', 'Baylink',
       'Blue & Gold Fleet', 'Boston Harbor Islands Ferries', 'Caltrain',
       'Capital Metro', 'Capitol Corridor Joint Powers Authority',
       'CAT  ', 'CATS', 'Chappaquiddick Ferry,Inc.',
       'Charm City Circulator', 'City of Seattle', 'CTA', 'DART',
       'DART, DCTA', 'DCTA, DART', 'Detroit Transportation Corporation',
       'DTOP ', 'GCRTA', 'Golden Gate Ferry',
       'Hampton Roads Transit (HRT)', 'Harbor Bay Ferry', 'HART', 'JTA ',
       'KAT', 'KCATA', 'King County Marine Division', 'Kitsap Transit',
       'Lane Transit District', 'Las Vegas Monorail',
       'Long Island Rail Road', 'MARTA',
       'Maryland Transit Administration', 'MATA ', 'MBTA', 'Metra',
       'Metro', 'METRO', 'Metrolink', 'Metro-North Railroad',
       'Metro St. Louis', 'Metro Transit', 'Miami Dade Transit', 'MTA',
       'MTS', 'New England Fast Ferry', 'NFTA-METRO',

In [143]:
from osmnx.utils_geo import bbox_from_point
# intersection_bbox = ",".join(
#     str(i)
#     for i in bbox_from_point(
#         point=(
#             43.592414,
#             -73.261698,
#         ),
#         dist=35,
#     )
# )
intersection_bbox = ",".join(
    str(i)
    for i in bbox_from_point(
        point=(
            40.759160,
            -73.988386,
        ),
        dist=35,
    )
)
intersection_bbox

'-73.9888015491527,40.75884523788258,-73.98797045084731,40.75947476211742'

In [144]:
from PIL import Image
from io import BytesIO # To read image from bytes

api_endpoint = "https://orthos.its.ny.gov/arcgis/rest/services/wms/2024/MapServer"
api_year = 2022

params = {"f": "json"}

source_extent = requests.get(api_endpoint, {"f": "json"}).json()["fullExtent"]
source_extent

server_bbox = list(
                transformer.transform(source_extent["xmin"], source_extent["ymin"])
            ) + list(
                transformer.transform(source_extent["xmax"], source_extent["ymax"])
            )

params = {
    "bbox": intersection_bbox,
    "size": "640,640",
    "f": "image",
    "bboxSR": str(4326),
}

r = requests.get(api_endpoint + "/export", params)
imageStream = BytesIO(r.content)
imageFile = Image.open(imageStream)
imageFile.getextrema()


(0, 255)

In [77]:
data_path = "C:/Users/ckaz3/Desktop/intersection-safety-infra-detection/data/interim/existing_stations_intersections.csv"
data = pd.read_csv(data_path, index_col=0)
data = data[data["Agency"] == "MTA"]

# for data_source in data_sources:
api_endpoint = "https://orthos.its.ny.gov/arcgis/rest/services/wms/2022/MapServer"
api_year = 2022

params = {"f": "json"}

r = requests.get(api_endpoint, params).json()

transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")
server_bbox = list(
    transformer.transform(r["fullExtent"]["xmin"], r["fullExtent"]["ymin"])
)

server_bbox.extend(
    transformer.transform(r["fullExtent"]["xmax"], r["fullExtent"]["ymax"])
)

In [76]:
import requests
from pyproj import Transformer
from tqdm import tqdm

data_path = "C:/Users/ckaz3/Desktop/intersection-safety-infra-detection/data/interim/existing_stations_intersections.csv"
data = pd.read_csv(data_path, index_col=0)
data = data[data["Agency"] == "MTA"]

# for data_source in data_sources:
api_endpoint = "https://orthos.its.ny.gov/arcgis/rest/services/wms/2022/MapServer"
api_year = 2022

params = {"f": "json"}

r = requests.get(api_endpoint, params).json()

transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")
server_bbox = list(
    transformer.transform(r["fullExtent"]["xmin"], r["fullExtent"]["ymin"])
)

server_bbox.extend(
    transformer.transform(r["fullExtent"]["xmax"], r["fullExtent"]["ymax"])
)

server_bbox_str = ",".join(str(i) for i in server_bbox)

for _, intersection in tqdm(data.iterrows(), total=len(data)):
    api_endpoint = "https://orthos.its.ny.gov/arcgis/rest/services/wms/2022/MapServer/export"

    # see if intersection is in bounding box
    if (
        (intersection["Inter_Latitude"] < server_bbox[0])
        or (intersection["Inter_Latitude"] > server_bbox[2])
        or (intersection["Inter_Longitude"] < server_bbox[1])
        or (intersection["Inter_Longitude"] > server_bbox[3])
    ):
        continue

    # TODO: if image has already been found

    # query image and save image as intersection_id__year
    bbox = ",".join(str(i) for i in utils_geo.bbox_from_point(
            point=(intersection["Inter_Latitude"], intersection["Inter_Longitude"]),
            dist=35,
        ))
    params = {
        "bbox": bbox,
        "size": "640,640",
        "f": "image",
        "bboxSR": str(4326)
    }

    r = requests.get(api_endpoint, params, stream=True)

    save_file_path = "C:/Users/ckaz3/Desktop/intersection-safety-infra-detection/data/raw/test_images/"

    with open(save_file_path 
              + str(intersection["Intersection_ID"]) 
              + "_" + str(api_year) + ".png", "wb") as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)

    # update table as true
    data.loc[data["Intersection_ID"] == intersection["Intersection_ID"], str(api_year)] = True

 61%|██████▏   | 5872/9559 [1:13:23<46:05,  1.33it/s]  


ReadTimeout: HTTPSConnectionPool(host='orthos.its.ny.gov', port=443): Read timed out. (read timeout=None)

In [208]:
data_path = "C:/Users/ckaz3/Desktop/intersection-safety-infra-detection/data/interim/existing_stations_intersections.csv"
data = pd.read_csv(data_path, index_col=0)

for i, row in data[data["Station Name"].str.startswith("42 St - P")].iterrows():
    print(
        str(
            utils_geo.bbox_from_point(
                point=(row["Inter_Latitude"], row["Inter_Longitude"]), dist=50
            )
        )
    )
    break

(np.float64(-73.98897944164224), np.float64(40.75870983983226), np.float64(-73.98779215835776), np.float64(40.75960916016775))
